In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter-one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

chain.run("Describe Victory Mansions")

'Victory Mansions is a building where Winston Smith resides. The hallway of Victory Mansions smells of boiled cabbage and old rag mats. The flat Winston lives in is seven flights up, with a poster of an enormous face with the caption "BIG BROTHER IS WATCHING YOU" at each landing. The flat has a telescreen that cannot be completely shut off, and Winston keeps his back turned to it for safety. The building itself is described as grimy, and the surroundings include rotting nineteenth-century houses with patched windows and sagging garden walls. The Ministry of Truth, a vast white structure, towers above the landscape from a distance.'